In [0]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50 as resnet


In [0]:
import cv2
import os
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout,Flatten,Conv2D,MaxPool2D,Dense, BatchNormalization
from tensorflow.keras.preprocessing import image

%matplotlib inline

In [3]:
! git clone https://github.com/AnnaFilipov93/flowers-analyzer

Cloning into 'flowers-analyzer'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 4409 (delta 8), reused 35 (delta 5), pack-reused 4369
Receiving objects: 100% (4409/4409), 368.70 MiB | 34.82 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (8711/8711), done.


In [0]:
# In case the model pre-trained
model=load_model('five_flowers_model.h5')

In [0]:
BATCH_SIZE=10
STEP=3500/BATCH_SIZE +1
VALID_STEPS=850/BATCH_SIZE +1

In [0]:
#prepering the data
train_image_gen=ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                    )

val_image_gen=ImageDataGenerator(rescale=1./255)


In [17]:
##~~reading the data ~~##
train_dir=os.path.join(os.getcwd(), 'flowers-analyzer/flowers/FLOWERS/train')
test_dir=os.path.join(os.getcwd(), 'flowers-analyzer/flowers/FLOWERS/test')

train_img=train_image_gen.flow_from_directory(train_dir,
                                         target_size=(224,224), batch_size=BATCH_SIZE,
                                       class_mode='categorical')


test_img=train_image_gen.flow_from_directory(test_dir,
                                       target_size=(224,224), batch_size=BATCH_SIZE, 
                                        class_mode='categorical')


Found 3501 images belonging to 5 classes.
Found 850 images belonging to 5 classes.


In [0]:
def resnet50_train():
    model = Sequential()

    model.add(resnet(include_top=False, pooling='avg', weights='imagenet'))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(5, activation='softmax'))

    model.layers[0].trainable = False
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit_generator(
        train_img,epochs=20, steps_per_epoch=STEP,
        validation_data=test_img, validation_steps=VALID_STEPS)

    return model


In [20]:
model = resnet50_train()

Epoch 1/20
351/351 [==============================] - 63s 180ms/step - loss: 1.3354 - accuracy: 0.6641 - val_loss: 0.8382 - val_accuracy: 0.7488
Epoch 2/20
351/351 [==============================] - 62s 175ms/step - loss: 0.7593 - accuracy: 0.7444 - val_loss: 0.6886 - val_accuracy: 0.7698
Epoch 3/20
351/351 [==============================] - 61s 174ms/step - loss: 0.6754 - accuracy: 0.7586 - val_loss: 0.6633 - val_accuracy: 0.7663
Epoch 4/20
351/351 [==============================] - 61s 174ms/step - loss: 0.6107 - accuracy: 0.7798 - val_loss: 0.5749 - val_accuracy: 0.8116
Epoch 5/20
351/351 [==============================] - 61s 174ms/step - loss: 0.5705 - accuracy: 0.7986 - val_loss: 0.6169 - val_accuracy: 0.7988
Epoch 6/20
351/351 [==============================] - 61s 174ms/step - loss: 0.5563 - accuracy: 0.8021 - val_loss: 0.6744 - val_accuracy: 0.7814
Epoch 7/20
351/351 [==============================] - 61s 174ms/step - loss: 0.5234 - accuracy: 0.8195 - val_loss: 0.6740 - val_ac

----- WITH BIAS : loss: 0.3830 - accuracy: 0.8700 - val_loss: 0.6431 - val_accuracy: 0.8012
----- WITHOUT BIAS : loss: 0.4067 - accuracy: 0.8603 - val_loss: 0.6585 - val_accuracy: 0.8000

In [0]:
from IPython.display import Image, display

import os, random
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
img_locations = []
for d in os.listdir("flowers-analyzer/flowers/FLOWERS/test/"):
    directory = "flowers-analyzer/flowers/FLOWERS/test/" + d
    sample = [directory + '/' + s for s in random.sample(
        os.listdir(directory), int(random.random()*10))]
    img_locations += sample

In [30]:
def read_and_prep_images(img_paths, img_height=224, img_width=224):
    imgs = [load_img(img_path, target_size=(224, 224)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    return preprocess_input(img_array)

random.shuffle(img_locations)
imgs = read_and_prep_images(img_locations)
predictions = model.predict_classes(imgs)
classes = dict((v,k) for k,v in train_img.class_indices.items())

for img, prediction in zip(img_locations, predictions):
    display(Image(img))
    print(classes[prediction])

Output hidden; open in https://colab.research.google.com to view.

In [0]:
'''def train_model():
    ###~~~building the model~~~###

    model=Sequential()

    #conv layer I
    model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(240,240,3),
                    activation='relu'))
    #pooling layer I
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    #conv layer II
    model.add(Conv2D(filters=64, kernel_size=(3,3),
                    activation='relu'))
    #pooling layer II
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    #conv layer III
    model.add(Conv2D(filters=64, kernel_size=(3,3),
                    activation='relu'))
    #pooling layer III
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    #conv layer IV
    model.add(Conv2D(filters=64, kernel_size=(3,3),
                    activation='relu'))
    #pooling layer IV
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    #conv layer V
    #model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
    #                 activation='relu'))
    #pooling layer V
    #model.add(MaxPool2D(pool_size=(2,2)))

    #conv layer VI
    #model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
    #                 activation='relu'))
    #pooling layer VI
    #model.add(MaxPool2D(pool_size=(2,2)))

    #flatten
    model.add(Flatten())

    #Dense layer
    model.add(Dense(512,activation='relu'))

    #dropout layer
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    #output layer
    model.add(Dense(5,activation='softmax'))

    model.summary()

    #compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    ###~~~training the model~~~###

    results=model.fit_generator(train_img,epochs=1, steps_per_epoch=1,
                                validation_data=test_img, validation_steps=27)

    return model'''

In [0]:
model.save('resnet50_classifier.h5')

---------- manual checking of the model performence ---------------

In [0]:
###~~~testing on the tests images~~~###

examples = ['FLOWERS//test//daisy/144076848_57e1d662e3_m.jpg',
            'FLOWERS//test//dandelion/14740350060_a489d9fa06.jpg',
            'FLOWERS//test/tulip/5633266048_4f4bfb2cf1_n.jpg',
            'FLOWERS//test/sunflower/21728822928_9f6817325a_n.jpg',
            'flowers//FLOWERS//test//rose/random_flower.jpg']


In [0]:
def pred (p, pred_acc):
    if pred_acc[0,p]<0.7:
        print("the model couldn't recognize it")
    elif p==0:
        print("the model recognized it as a daisy")
    elif p==1:
         print("the model recognized it as a dandelion")
    elif p==2:
          print("the model recognized it as a rose")
    elif p==3:
          print("the model recognized it as a sunflower")       
    elif p==4:
          print("the model recognized it as a tulip")     
            

In [0]:
def predict_flower(path):
  flower=cv2.imread(path)
  flower=cv2.cvtColor(flower,cv2.COLOR_BGR2RGB)
  plt.imshow(flower)

  flower_file=path
  flower_img=image.load_img(flower_file, target_size=(240,240))
  flower_img=image.img_to_array(flower_img)
  flower_img=np.expand_dims(flower_img,axis=0)
  flower_img=flower_img/255

  p=model.predict_classes(flower_img)
  pred(p)

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()